In [31]:
import numpy as np
import pandas as pd
import time
from nltk import *
from stop_words import get_stop_words

# import string
from sklearn import feature_extraction

In [32]:
class spanish_tweet_tokenizer_with_date:
    """Recibe ts o df con dos columnas: fecha en formato twitter (como índice) y el texto de los tweets"""
    """Receives ts or df with two columns: date in twitter format (as index) and tweets text."""
    def __init__(self,tweets):
        # Asumo que la información no tiene estorbos (nans, cosas distintas a fechas en el índice...)
        # I assume info hasn't any hindrance
        try:
            tweets.stack()
        except:
            pass
        try:
            date = pd.to_datetime([time.strftime('%Y-%m-%d %H:%M:%S', j) 
                                   for j in [time.strptime(i,'%a %b %d %H:%M:%S +0000 %Y') for i in tweets.index]])
        except:
            date = pd.to_datetime(tweets.index)
        tokenizer = tokenize.simple.SpaceTokenizer() #para evitar problemas con acentuación
        self.date_corrected = pd.Series(tweets.values,
                                        index=date)
        
        tweets = tweets.str.lower()
        tokens = []
        dates  = []
        tknd_tweets = [] #aquí almacenaré tweets ya tokenizados
        try:
            decoded_tweets = [x.decode('utf-8') for x in tweets.astype('string')]
        except IndexError:
            decoded_tweets = [x.decode('utf-8') for x in tweets.iloc[:,1].astype('string')]
        for t in xrange(len(decoded_tweets)):
            tokened = tokenizer.tokenize(decoded_tweets[t])
            tknd_tweets.append(tokened)
            tweet_date = date[t]
            for word in xrange(len(tokened)):
                dates.append(tweet_date)
                if len(tokened[word])<=1:
                    tokens.append(tokened[word])
                elif tokened[word][-1] in [sign.decode('utf-8') for sign in ['?','!',',',':','.','-','"',')']]:
                    tokens.append(tokened[word][:-1])
                elif tokened[word][0] in [sign.decode('utf-8') for sign in ['¿','¡',',',':','.','-','"', '(']]:
                    tokens.append(tokened[word][1:])
                else:
                    tokens.append(tokened[word].lower())
        frame = pd.DataFrame(np.asarray(tokens).reshape(len(tokens)),
                             columns=['tokens'],
                             index=pd.to_datetime(dates))
        self.tokened_tweets_df = frame
        self.tokened_tweets_ts = frame.stack()
        self.tokened_tweets = frame.values
        self.n_tokens = frame.shape[0]
        self.n_tweets = tweets.shape[0]
        
        # Agregaré en automático el cómputo sin stop_words
        stop_words_sp = get_stop_words('spanish')
        #algunas palabras que no incluye
        stop_words_sp.append(' ')
        stop_words_sp.append('')
        stop_words_sp.append('-')
        stop_words_sp.append(u'si')
        stop_words_sp.append(u'sí')
        ix_no_sw    = np.in1d(self.tokened_tweets, stop_words_sp)!=1 #stopwords
        frame_no_sw = self.tokened_tweets_ts[ix_no_sw]

        self.tokened_tweets_no_sw = frame_no_sw
        self.n_tokens_no_sw       = frame_no_sw.shape[0]
    
    def n_tokens(self):
        return self.n_words
    
    def n_tweets(self):
        return self.n_tweets
    
    def lexical_diversity(self):
        distinct_words = len(set([word for word in self.tokened_tweets_ts]))
        total_words    = self.tokened_tweets.shape[0]
        
        lex_div = 1.*distinct_words/total_words
        return lex_div
    
    def avg_words_p_tweet(self):
        avg_words = 1.*self.n_tokens/self.n_tweets
        return avg_words
    
    def tokened_tweets_no_sw(self):
        return self.tokened_tweets_no_sw
        
    def n_tokens_no_sw(self):
        return self.n_tokens_no_sw
    
    def lexical_diversity_no_sw(self):
        distinct_words = len(set([word for word in self.tokened_tweets_no_sw]))
        total_words    = self.n_tokens_no_sw

        lex_div = 1.*distinct_words/total_words
        return lex_div

    def avg_words_p_tweet_no_sw(self):
        avg_words = 1.*self.n_tokens_no_sw/self.n_tweets
        return avg_words
    
    def lookfor(self, words_to_look, include_sw):
        """Returns a filtered ts that includes only words_to_look, over ts with or without stopwords"""
        if include_sw==True:
            frame = self.tokened_tweets_no_sw
            n_obs = self.n_tokens_no_sw
        else:
            frame = self.tokened_tweets_ts
            n_obs = self.n_tokens
        
        temp_ix = np.ones((1,n_obs))
        
        for w in words_to_look:
            temp_ix = np.concatenate((temp_ix,frame.str.match(w).values.reshape(1,n_obs)),
                                     axis=0)
        ix    = pd.DataFrame(temp_ix[1:,:])
        
        final = frame.loc[(ix.mean()>0).values]
        
        return final
    def lookfor_intweets(self, words_to_look):
        """Retrieves tweets that contain any of the given words."""
        frame = self.date_corrected.str.lower()
        n_obs = frame.shape[0]
        
        temp_ix = np.ones((1,n_obs))
        
        for w in words_to_look:
            temp_ix = np.concatenate((temp_ix,frame.str.contains(w).values.reshape(1,n_obs)),
                                     axis=0)
        ix    = pd.DataFrame(temp_ix[1:,:])
        final = frame.loc[(ix.sum()>0).values]
        
        return final

In [33]:
data_dict  = {0:'banamex',
              1:'SantanderMx',
              2:'ScotiabankMX',
              3:'BBVABancomer'}
n_accounts = len(data_dict)

In [69]:
# Palabras promocionales y de exp al cliente
#Creo que la forma más eficiente es identificar la fecha e identificar ese tweet
capta   = [u'vista',u'plazo',u'interés',u'intereses',u'tasa',
           u'débito',u'debito',u'depósito',u'deposit',u'cuenta']

credit  = [u'crédito',u'credit', u'hipoteca',u'auto',u'nómina',u'personal',
           u'tarjeta',u'tdc',u'arrendamiento',u'msi']

invest  = [u'inversi',u'pagaré',u'dinero',u'bolsa',
           u'invierte',u'fondo',u'ahorr',u'invertir']

tdc     = [u'tdc',u'puntos',u'cashback',u'meses',u'balance',
           u'saldo',u'transfer',u'cash',u'msi',u'tarjeta',
           u'compras',u'paga',u'pagos',]

exp_clt = [u'sucursal',u'servicio',u'atención',u'atend',u'queja']

seguro  = [u'seguro',u'inmobili',u'médico',u'gastos',u'vida']

general = [u'comision',u'comisión',u'promo',u'descuento',
           u'regístrate',u'conoce',u'tyc', u'términos', 
           u'terminos', u'condici',u'vívelo',u'experienc',
           u'programa',u'vive',u'contrat',u'adquiere',
           u'aprovecha',u'vigencia',u'deuda',u'operaci',
           u'liquida',u'gratis',u'gratui',u'premia',u'express',
           u'compras',u'ventas',u'ingresa',u'especial',u'membresía',
           u'empresarial']

viajes  = [u'viaj',u'destino',u'vuela',u'planea',
           u'anticipación',u'avión']

shows   = [u'boleto',u'preven',u'concierto',u'exclusiva',
           u'moto',u'vehículo',u'vehicular',u'cine',u'ticket'
           u'asiste',u'experiencia']

autos   = [u'carro',u'coche','auto']

casa    = [u'casa',u'hogar',u'inmobiliario',u'vivienda']

mobile  = [u'internet',u'móvil',u'banca',u'servicio',
           u'celular', u'localizador',u'telefónic',
           u'descarga',u'línea',u'sms',u'app',u'wallet',
           u'mobile',u'consulta',u'laptop',u'sms',u'internet',
           u'retiro',u'cargo']

mkt_words = capta+credit+invest+tdc+exp_clt+seguro+general+viajes+shows+autos+casa+mobile


divisas  = [u'dólar', u'peso', u'dolar',u'indicador',
            u'tipo de cambio',u'tipodecambio']
vacantes = [u'vacante',u'linkedin',u'trabajo']
talleres = [u'taller',u'finanzas']

inf_words = divisas+vacantes

other_news = [u'sabíasque',u'cultura']


clt_words = [u'hola', u'favor', u'contacto', u'saludos', u'nombre', 
             u'dm', u'report', u'sentimos', u'=', u'llamada', u'agrade',
             u'gracias', u'caso', u'atendemos',u'lamenta',u'molestia',
             u'buenas',u'buenos',u'md']

In [70]:

mkt_categos     = {u'Captación':capta, 
                   u'Crédito':credit,
                   u'Inversiones':invest, 
                   'Experiencia al cliente':exp_clt,
                   'TdC':tdc, 
                   'Seguro':seguro, 
                   'General':general, 
                   'Viajes':viajes, 
                   'Shows':shows, 
                   'Autos':autos, 
                   'Casa':casa, 
                   u'Móvil o Internet':mobile}

mkt_categos_tags = [u'Captación',
                    u'Crédito',
                    u'Inversiones',
                    u'TdC',
                    'Experiencia al cliente',
                    'Seguro', 
                    'General', 
                    'Viajes', 
                    'Shows', 
                    'Autos', 
                    'Casa', 
                    u'Móvil o Internet'] 

In [74]:
# Cómo se publica de acuerdo a cada subcategoría de mkt (desde enero)
for b in xrange(n_accounts):
    print '\n',data_dict[b]
    
    datum1 = pd.read_csv('tweets_'+data_dict[b]+'.csv')
    print datum1.shape
    datum1.drop([u'Unnamed: 0'], axis=1, inplace=True)
    datum1.drop_duplicates(inplace=True)
    datum1.drop(0, inplace=True)
    datum1.set_index(datum1.Fecha, inplace=True)
    datum1 = datum1.Tweet
    
    datum2 = pd.read_csv('tweets_'+data_dict[b]+'sinceMarch.csv')
    print datum2.shape
    datum2.drop([u'Unnamed: 0'], axis=1, inplace=True)
    datum2.drop_duplicates(inplace=True)
    datum2.drop(0, inplace=True)
    datum2.set_index(datum2.Fecha, inplace=True)
    datum2 = datum2.Tweet
    
    data = pd.concat([datum1,datum2])
    del datum1, datum2
    print data.shape
    data.drop_duplicates(inplace=True)
    print data.shape
    
    data.drop_duplicates(inplace=True)
    tokenized  = spanish_tweet_tokenizer_with_date(data)
    #Creo el índice
    ix_jan = tokenized.date_corrected.index>pd.to_datetime('2015-12-31 23:59:59')
    #Creo la ts que satisface ese índice
    jan_tweets = tokenized.date_corrected[ix_jan]
    tokenized  = spanish_tweet_tokenizer_with_date(jan_tweets)
    n_tweets = tokenized.n_tweets
    #Proporción que dedican a cada cosa...
    clt_frame  = tokenized.lookfor_intweets(clt_words)
    for cat in mkt_categos_tags:
        print cat
        mkt_frame  = tokenized.lookfor_intweets(mkt_categos[cat])
        mkt_tweets = mkt_frame.shape[0]
        base = np.zeros((1,mkt_tweets))
        print '-----', mkt_tweets, mkt_frame.shape
        for word in clt_words:
            temp = mkt_frame.str.contains(word).values.reshape(1,mkt_tweets)
            base = np.concatenate((base,temp),axis=0)
        ix_pure_mkt = pd.DataFrame(base[1:,:]).mean()==0
        mkt_frame = mkt_frame[ix_pure_mkt.values]

        mkt_num   = pd.Series([1.0 for i in mkt_frame],
                              index=mkt_frame.index-pd.to_timedelta('6H'))

        mkt_num.fillna(0,inplace=True)
        print mkt_num.shape
        try:
            mkt_num.to_csv('mkt_frame_'+data_dict[b]+'_'+cat+'.csv')
        except:
            print 'No se pudo: ', mkt_num.sum()



banamex
(3201, 24)
(3201, 18)
(6400L,)
(3601L,)
Captación
----- 13 (13L,)
(12L,)
Crédito
----- 54 (54L,)
(50L,)
Inversiones
----- 10 (10L,)
(10L,)
TdC
----- 97 (97L,)
(91L,)
Experiencia al cliente
----- 6 (6L,)
(6L,)
Seguro
----- 18 (18L,)
(17L,)
General
----- 206 (206L,)
(190L,)
Viajes
----- 14 (14L,)
(14L,)
Shows
----- 98 (98L,)
(85L,)
Autos
----- 5 (5L,)
(5L,)
Casa
----- 4 (4L,)
(3L,)
Móvil o Internet
----- 86 (86L,)
(83L,)

SantanderMx
(3201, 24)
(3201, 18)
(6400L,)
(6396L,)
Captación
----- 523 (523L,)
(108L,)
Crédito
----- 374 (374L,)
(193L,)
Inversiones
----- 40 (40L,)
(25L,)
TdC
----- 361 (361L,)
(202L,)
Experiencia al cliente
----- 911 (911L,)
(103L,)
Seguro
----- 51 (51L,)
(35L,)
General
----- 361 (361L,)
(184L,)
Viajes
----- 50 (50L,)
(37L,)
Shows
----- 85 (85L,)
(65L,)
Autos
----- 59 (59L,)
(36L,)
Casa
----- 50 (50L,)
(41L,)
Móvil o Internet
----- 305 (305L,)
(82L,)

ScotiabankMX
(3200, 24)
(3199, 18)
(6397L,)
(4524L,)
Captación
----- 106 (106L,)
(50L,)
Crédito
----- 177 (1

In [62]:
# Hrs en que publican desde enero
for b in xrange(n_accounts-2):
    print '\n',data_dict[b]
    
    datum1 = pd.read_csv('tweets_'+data_dict[b]+'.csv')
    print datum1.shape
    datum1.drop([u'Unnamed: 0'], axis=1, inplace=True)
    datum1.drop_duplicates(inplace=True)
    datum1.drop(0, inplace=True)
    datum1.set_index(datum1.Fecha, inplace=True)
    datum1 = datum1.Tweet
    
    datum2 = pd.read_csv('tweets_'+data_dict[b]+'sinceMarch.csv')
    print datum2.shape
    datum2.drop([u'Unnamed: 0'], axis=1, inplace=True)
    datum2.drop_duplicates(inplace=True)
    datum2.drop(0, inplace=True)
    datum2.set_index(datum2.Fecha, inplace=True)
    datum2 = datum2.Tweet
    
    data = pd.concat([datum1,datum2])
    del datum1, datum2
    print data.shape
    data.drop_duplicates(inplace=True)
    print data.shape
    
    tokenized  = spanish_tweet_tokenizer_with_date(data)
    #Creo el índice
    ix_jan = tokenized.date_corrected.index>pd.to_datetime('2015-12-31 23:59:59')
    #Creo la ts que satisface ese índice
    jan_tweets = tokenized.date_corrected[ix_jan]
    tokenized  = spanish_tweet_tokenizer_with_date(jan_tweets)
    n_tweets = tokenized.n_tweets
    #Proporción que dedican a cada cosa...
    clt_frame  = tokenized.lookfor_intweets(clt_words)
    mkt_frame  = tokenized.lookfor_intweets(mkt_words)
    mkt_tweets = mkt_frame.shape[0]
    base = np.zeros((1,mkt_tweets))
    for word in clt_words:
        temp = mkt_frame.str.contains(word).values.reshape(1,mkt_tweets)
        base = np.concatenate((base,temp),axis=0)
    ix_pure_mkt = pd.DataFrame(base[1:,:]).mean()==0
    mkt_frame = mkt_frame[ix_pure_mkt.values]

    mkt_num   = pd.Series([1.0 for i in mkt_frame],
                          index=mkt_frame.index-pd.to_timedelta('6H'))
    
    mkt_num.fillna(0,inplace=True)
    mkt_num.to_csv('mkt_frame_'+data_dict[b]+'.csv')
    
    morn = mkt_num.between_time('00:00', '9:00',include_end=False).resample('1D',how='sum')
    aft  = mkt_num.between_time('9:00', '18:00',include_end=False).resample('1D',how='sum')
    noon = mkt_num.between_time('18:00','00:00',include_end=False).resample('1D',how='sum')
    
    morn.fillna(0,inplace=True)
    aft.fillna(0,inplace=True)
    noon.fillna(0,inplace=True)
    
    morn_avg = morn.mean()
    aft_avg  = aft.mean()
    noon_avg = noon.mean()
    
    if morn_avg>0:
        0
    else:
        morn_avg = 0
    print 'daily avg mkt tweets 00:00 - <9:00:    ',morn_avg
    print 'daily avg mkt tweets 9:00 - <18:00:    ',aft_avg
    print 'daily avg mkt tweets 18:00 - <00:00:   ',noon_avg


banamex
(3201, 24)
(3201, 18)
(6400L,)
(3601L,)
daily avg mkt tweets 00:00 - <9:00:     0
daily avg mkt tweets 9:00 - <18:00:     2.54621848739
daily avg mkt tweets 18:00 - <00:00:    0.132743362832

SantanderMx
(3201, 24)
(3201, 18)
(6400L,)
(6396L,)
daily avg mkt tweets 00:00 - <9:00:     0.731707317073
daily avg mkt tweets 9:00 - <18:00:     3.20325203252
daily avg mkt tweets 18:00 - <00:00:    0.716666666667


In [75]:
# Histórico bbva y banamex (por categoría y en general)
for b in ['banamex','BBVABancomer']:
    print '\n',b
    
    datum1 = pd.read_csv('tweets_'+b+'.csv')
    print datum1.shape
    datum1.drop([u'Unnamed: 0'], axis=1, inplace=True)
    datum1.drop_duplicates(inplace=True)
    datum1.drop(0, inplace=True)
    datum1.set_index(datum1.Fecha, inplace=True)
    datum1 = datum1.Tweet
    
    datum2 = pd.read_csv('tweets_'+b+'sinceMarch.csv')
    print datum2.shape
    datum2.drop([u'Unnamed: 0'], axis=1, inplace=True)
    datum2.drop_duplicates(inplace=True)
    datum2.drop(0, inplace=True)
    datum2.set_index(datum2.Fecha, inplace=True)
    datum2 = datum2.Tweet
    
    data = pd.concat([datum1,datum2])
    del datum1, datum2
    print data.shape
    data.drop_duplicates(inplace=True)
    print data.shape
    
    
    tokenized  = spanish_tweet_tokenizer_with_date(data)
    clt_frame  = tokenized.lookfor_intweets(clt_words)
    mkt_frame  = tokenized.lookfor_intweets(mkt_words)
    mkt_tweets = mkt_frame.shape[0]
    base = np.zeros((1,mkt_tweets))
    for word in clt_words:
        temp = mkt_frame.str.contains(word).values.reshape(1,mkt_tweets)
        base = np.concatenate((base,temp),axis=0)
    ix_pure_mkt = pd.DataFrame(base[1:,:]).mean()==0
    mkt_frame = mkt_frame[ix_pure_mkt.values]

    mkt_num   = pd.Series([1.0 for i in mkt_frame],
                          index=mkt_frame.index-pd.to_timedelta('6H'))
    print mkt_num.index.min()
    mkt_num.fillna(0,inplace=True)
    mkt_num.to_csv('mkt_frame_historic_'+b+'.csv')
    for cat in mkt_categos_tags:
        print cat
        mkt_frame  = tokenized.lookfor_intweets(mkt_categos[cat])
        mkt_tweets = mkt_frame.shape[0]
        base = np.zeros((1,mkt_tweets))
        for word in clt_words:
            temp = mkt_frame.str.contains(word).values.reshape(1,mkt_tweets)
            base = np.concatenate((base,temp),axis=0)
        ix_pure_mkt = pd.DataFrame(base[1:,:]).mean()==0
        mkt_frame = mkt_frame[ix_pure_mkt.values]

        mkt_num   = pd.Series([1.0 for i in mkt_frame],
                              index=mkt_frame.index-pd.to_timedelta('6H'))

        mkt_num.fillna(0,inplace=True)
        print mkt_num.shape
        mkt_num.to_csv('mkt_frame_historic_'+b+'_'+cat+'.csv')


banamex
(3201, 24)
(3201, 18)
(6400L,)
(3601L,)
2014-09-07 11:00:05
Captación
(118L,)
Crédito
(351L,)
Inversiones
(91L,)
TdC
(603L,)
Experiencia al cliente
(29L,)
Seguro
(128L,)
General
(779L,)
Viajes
(79L,)
Shows
(355L,)
Autos
(62L,)
Casa
(41L,)
Móvil o Internet
(258L,)

BBVABancomer
(3200, 24)
(3200, 18)
(6398L,)
(3614L,)
2014-10-14 15:12:28
Captación
(152L,)
Crédito
(395L,)
Inversiones
(474L,)
TdC
(737L,)
Experiencia al cliente
(136L,)
Seguro
(289L,)
General
(924L,)
Viajes
(122L,)
Shows
(37L,)
Autos
(98L,)
Casa
(94L,)
Móvil o Internet
(513L,)


In [84]:
hj = mkt_num.resample('1D',how='sum')
hj.index.max()

Timestamp('2016-05-02 00:00:00', offset='D')